In [10]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext
conf = SparkConf()
conf.set("spark.master","local[3]")
conf.set("spark.app.name","sparkapp")
sc = SparkContext(conf=conf)
sqlc = SQLContext(sc)

In [11]:
sqlc

In [12]:
df1 = sqlc.read.format("csv").option("header","True").option("inferSchema","True").load("/home/ubuntu/Datasets/cancerdata.csv")
df1.show(5)

+------+-----+----+----+--------+-----------------+---------+---------+--------+-----+---------+--------+
|    ID| Name| Age| Sex|   State|         Symptoms|Diagnosis|   Cancer|CancerSc|Stage|Treatment|Survival|
+------+-----+----+----+--------+-----------------+---------+---------+--------+-----+---------+--------+
|  null| null|null|null|    null|             null|     null|     null|    null| null|     null|    null|
|100001|David|  45|   M| ALAMAMA|RED ITCHY PATCHES|   BIOPSY|MALIGNANT|    SKIN|    1|RESECTION|     YES|
|100002| John|  56|   M|  ALASKA|      BLOOD COUGH| PET SCAN|MALIGNANT|THORACIC|    2|  SURGERY|     YES|
|100003| Paul|  65|   M| ARIZONA|RED ITCHY PATCHES|   BIOPSY|MALIGNANT|    SKIN|    3|RESECTION|      NO|
|100004| Mark|  35|   M|ARKANSAS|      BLOOD COUGH| PET SCAN|MALIGNANT|THORACIC|    3|  SURGERY|      NO|
+------+-----+----+----+--------+-----------------+---------+---------+--------+-----+---------+--------+
only showing top 5 rows



In [18]:
df1.createOrReplaceTempView("temptable")
df1.write.saveAsTable("permtable")
sqlc.sql("select * from temptable limit 5").show()
sqlc.sql("select * from permtable limit 5").show()
df1.write.format("json").saveAsTable("jsontable")

+------+-----+----+----+--------+-----------------+---------+---------+--------+-----+---------+--------+
|    ID| Name| Age| Sex|   State|         Symptoms|Diagnosis|   Cancer|CancerSc|Stage|Treatment|Survival|
+------+-----+----+----+--------+-----------------+---------+---------+--------+-----+---------+--------+
|  null| null|null|null|    null|             null|     null|     null|    null| null|     null|    null|
|100001|David|  45|   M| ALAMAMA|RED ITCHY PATCHES|   BIOPSY|MALIGNANT|    SKIN|    1|RESECTION|     YES|
|100002| John|  56|   M|  ALASKA|      BLOOD COUGH| PET SCAN|MALIGNANT|THORACIC|    2|  SURGERY|     YES|
|100003| Paul|  65|   M| ARIZONA|RED ITCHY PATCHES|   BIOPSY|MALIGNANT|    SKIN|    3|RESECTION|      NO|
|100004| Mark|  35|   M|ARKANSAS|      BLOOD COUGH| PET SCAN|MALIGNANT|THORACIC|    3|  SURGERY|      NO|
+------+-----+----+----+--------+-----------------+---------+---------+--------+-----+---------+--------+

+------+-----+----+----+--------+------------

In [15]:
# df1.write.partitionBy("Sex").saveAsTable("ptable1")
df1.write.format("orc").partitionBy("Sex","State").mode("append").saveAsTable("ptable1")

In [16]:
df1.write.bucketBy(4,"ID").saveAsTable("btable1")

In [19]:
sqlc.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|  btable1|      false|
| default|jsontable|      false|
| default|permtable|      false|
| default|  ptable1|      false|
|        |temptable|       true|
+--------+---------+-----------+



In [23]:
df1.write.partitionBy("Age").bucketBy(3,"Sex").mode("overwrite").saveAsTable("pbtable")

In [26]:
df1.filter("Sex!='M' and Sex!='F'").show()

+---+----+---+---+-----+--------+---------+------+--------+-----+---------+--------+
| ID|Name|Age|Sex|State|Symptoms|Diagnosis|Cancer|CancerSc|Stage|Treatment|Survival|
+---+----+---+---+-----+--------+---------+------+--------+-----+---------+--------+
+---+----+---+---+-----+--------+---------+------+--------+-----+---------+--------+



In [39]:
from pyspark.sql import Row
r1 = Row(ID=101,Name="xyz",Age=20,Sex="",State="xyz",Symptoms="xyz",Diagnosis="xyz",Cancer="xyz",CancerSc="xyz",Stage="xyz",Treatment="xyz",Survival="xyz")
df2 = sqlc.createDataFrame([r1])
# df2.show()

# df2.filter("Sex!='M' and Sex!='F'").show()

df3 = df1.union(df2)
# df3.show(5)


+----+
| Sex|
+----+
|   F|
|null|
|   M|
| xyz|
+----+



In [41]:
df3.select("Sex").distinct().show()

+----+
| Sex|
+----+
|   F|
|null|
|   M|
| xyz|
+----+



In [42]:
df3.write.partitionBy("Age").bucketBy(3,"Sex").mode("overwrite").saveAsTable("pbtable2")

In [44]:
df3.select("Sex").where("Age=20").distinct().show()

+---+
|Sex|
+---+
|  F|
|  M|
+---+



In [48]:
from pyspark.sql.functions import *
df3.groupBy("Sex").agg(count("*")).show()

+----+--------+
| Sex|count(1)|
+----+--------+
|   F|    6000|
|null|       1|
|   M|    4000|
| xyz|       1|
+----+--------+

